In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [2]:
dataTrain=pd.read_csv("../data/allAtt_onehot_large_train.csv")
dataTest=pd.read_csv("../data/allAtt_onehot_large_test.csv")
print(dataTrain.head())
print(dataTrain.shape)

   FTR  HTGS  ATGS  HTGC  ATGC  HTP  ATP  HM1  HM2  HM3  ...  ATWinStreak5  \
0    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
1    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
2    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
3    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
4    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   

   ATLossStreak3  ATLossStreak5  HTGD  ATGD  DiffPts  DiffFormPts  DiffLP  \
0              0              0   0.0   0.0      0.0          0.0   -11.0   
1              0              0   0.0   0.0      0.0          0.0     2.0   
2              0              0   0.0   0.0      0.0          0.0     2.0   
3              0              0   0.0   0.0      0.0          0.0   -17.0   
4              0              0   0.0   0.0      0.0          0.0     4.0   

   final1  final2  
0     1.0     0.0  
1     1.0     0.0  
2     0.

In [3]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FTR            1860 non-null   int64  
 1   HTGS           1860 non-null   float64
 2   ATGS           1860 non-null   float64
 3   HTGC           1860 non-null   float64
 4   ATGC           1860 non-null   float64
 5   HTP            1860 non-null   float64
 6   ATP            1860 non-null   float64
 7   HM1            1860 non-null   int64  
 8   HM2            1860 non-null   int64  
 9   HM3            1860 non-null   int64  
 10  HM4            1860 non-null   int64  
 11  AM1            1860 non-null   int64  
 12  AM2            1860 non-null   int64  
 13  AM3            1860 non-null   int64  
 14  AM4            1860 non-null   int64  
 15  HTWinStreak3   1860 non-null   int64  
 16  HTWinStreak5   1860 non-null   int64  
 17  HTLossStreak3  1860 non-null   int64  
 18  HTLossSt

In [4]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 1).
# Each input sequence will be of size (28, 1).
input_dim = 27

units = 64
output_size = 2  # labels are from Win or Loss

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(input_dim,1))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(input_dim,1)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

In [5]:
x_train, y_train = dataTrain.iloc[:,1:28].values,dataTrain.iloc[:,28:].values
x_train=np.reshape(x_train,(1860,27,1))
x_test, y_test = dataTest.iloc[:,1:28].values,dataTest.iloc[:,28:].values
x_test=np.reshape(x_test,(800,27,1))

In [6]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer="Adam",
    metrics=["categorical_accuracy"],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 17,282
Trainable params: 17,154
Non-trainable params: 128
_________________________________________________________________


In [7]:
model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=10
)

Epoch 1/10
30/30 [==============================] - 7s 43ms/step - loss: 0.6180 - categorical_accuracy: 0.6742 - val_loss: 0.6840 - val_categorical_accuracy: 0.5263
Epoch 2/10
30/30 [==============================] - 1s 21ms/step - loss: 0.6074 - categorical_accuracy: 0.6677 - val_loss: 0.6792 - val_categorical_accuracy: 0.5288
Epoch 3/10
30/30 [==============================] - 1s 20ms/step - loss: 0.6077 - categorical_accuracy: 0.6672 - val_loss: 0.6749 - val_categorical_accuracy: 0.5263
Epoch 4/10
30/30 [==============================] - 1s 21ms/step - loss: 0.6060 - categorical_accuracy: 0.6677 - val_loss: 0.6696 - val_categorical_accuracy: 0.5825
Epoch 5/10
30/30 [==============================] - 1s 23ms/step - loss: 0.6111 - categorical_accuracy: 0.6667 - val_loss: 0.6608 - val_categorical_accuracy: 0.6375
Epoch 6/10
30/30 [==============================] - 1s 20ms/step - loss: 0.6052 - categorical_accuracy: 0.6747 - val_loss: 0.6570 - val_categorical_accuracy: 0.6237
Epoch 7/10